# Setting

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.utils import shuffle

import torch
from datasets import Dataset, DatasetDict
from transformers import AdamW, EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification
from kobert_tokenizer import KoBERTTokenizer


import wandb
import warnings
warnings.filterwarnings('ignore')

/home/k-cat/anaconda3/envs/lth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-26 17:29:17.930895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 17:29:18.016298: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 17:29:18.036562: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to

In [ ]:
wandb.init(project='grooming', name='kobert_finetuning')

wandb: Currently logged in as: oiehhun (oiehhun-yonsei-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# Final Dataset

In [ ]:
train_data = pd.read_csv('../data/text_data/train_data.csv')
valid_data = pd.read_csv('../data/text_data/valid_data.csv')
test_data = pd.read_csv('../data/text_data/test_data.csv')

train_data = shuffle(train_data, random_state=42).reset_index(drop=True)

# Model Load

In [27]:
# BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenizer

In [7]:
def tokenize_function(data):
    return tokenizer(
        data['text'],
        add_special_tokens=False,   # 이미 [CLS], [SEP] 추가됨
        max_length=512,             # 문장 최대 길이
        truncation=True,            # 문장이 max_length보다 길면 자름
        padding=True                # 문장이 max_length보다 짧으면 padding
    )

# 첫 번째 텍스트에 대해 토큰화 수행
tokenized_output = tokenize_function(train_data.iloc[0])

# 결과 출력
print(f"input_ids: {tokenized_output['input_ids']}")
print(f"token_type_ids: {tokenized_output['token_type_ids']}")
print(f"attention_mask: {tokenized_output['attention_mask']}")
print(f"decoded tokens: {tokenizer.convert_ids_to_tokens(tokenized_output['input_ids'])}")

input_ids: [2, 1185, 5400, 1457, 7835, 2125, 5898, 3156, 7083, 4204, 5405, 6855, 54, 3, 3097, 46, 1221, 5850, 1406, 2123, 6079, 2628, 2233, 2874, 2355, 5683, 2874, 3278, 55, 7347, 517, 6357, 6844, 54, 3, 1469, 5330, 2964, 1370, 5439, 3945, 5595, 3219, 7788, 517, 6751, 7083, 3169, 7303, 46, 1370, 2962, 4196, 3868, 55, 3, 2267, 5550, 3394, 5474, 6896, 1723, 7864, 6855, 54, 3, 3166, 5405, 6855, 46, 880, 1907, 54, 3, 3135, 5724, 517, 364, 365, 364, 3, 3135, 5724, 46, 3942, 4307, 258, 3, 1100, 6797, 54, 2267, 5550, 4045, 4384, 6896, 517, 6989, 6855, 54, 3, 3166, 5405, 6855, 46, 1370, 4299, 995, 5468, 3991, 2011, 3868, 54, 1469, 5330, 3301, 3879, 4205, 54, 3, 3093, 3166, 5405, 6855, 46, 2149, 6812, 46, 1457, 2267, 7848, 7788, 517, 6751, 7318, 3155, 5, 1370, 5859, 517, 5540, 54, 3, 3097, 6844, 1100, 6797, 46, 1457, 2705, 7788, 3868, 54, 3, 1406, 4996, 1469, 2123, 2224, 921, 1267, 5876, 54, 3, 4102, 7096, 6844, 258, 1375, 1469, 5330, 2186, 6488, 5771, 1435, 1174, 7396, 5400, 4930, 905, 832, 15

# Dataset

In [8]:
# 검증(Vaildation) 데이터셋 분리
print(train_data.shape, valid_data.shape, test_data.shape)

(8250, 2) (1034, 2) (1024, 2)


In [9]:
# Dataset 생성
train_dataset = Dataset.from_pandas(train_data) # pandas DataFrame -> Hugging Face Dataset 형식으로 변환
valid_dataset = Dataset.from_pandas(valid_data)
test_dataset = Dataset.from_pandas(test_data)

datasets = DatasetDict({'train': train_dataset, 'valid': valid_dataset, 'test': test_dataset}) # train, valid, test 데이터셋을 묶어서 저장
tokenized_datasets = datasets.map(tokenize_function, batched=True) # train, vaild, test 데이터셋에 tokenize_function 적용

Map:   0%|          | 0/8250 [00:00<?, ? examples/s]

Map: 100%|██████████| 1024/1024 [00:00<00:00, 1577.12 examples/s]


In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8250
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1034
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1024
    })
})

# Train

In [ ]:
# 모델 저장 경로 설정
model_save_path = '../models/model_save'

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir=model_save_path,                 # 학습 결과 저장 경로
    report_to='wandb',                          # wandb 사용
    num_train_epochs=15,                        # 학습 epoch 설정
    per_device_train_batch_size=32,             # train batch_size 설정
    per_device_eval_batch_size=32,              # test batch_size 설정
    logging_dir=model_save_path+'/logs',        # 학습 log 저장 경로
    logging_steps=100,                          # 학습 log 기록 단위
    save_total_limit=2,                         # 학습 결과 저장 최대 개수
    evaluation_strategy="epoch",                # 매 epoch마다 평가 실행
    save_strategy="epoch",                      # 매 epoch마다 모델 저장
    load_best_model_at_end=True,                # 가장 성능이 좋은 모델을 마지막에 load
)

# 최적화 알고리즘(optimizer) 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# # 스케줄러(scheduler) 설정
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=0,
#     num_training_steps=len(tokenized_datasets['train']) * training_args.num_train_epochs
# )

In [43]:
# 성능 평가 지표 설정(binary classification)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [44]:
# Trainer 생성
trainer = Trainer(
    model=model, 
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
# 모델 학습
trainer.train()

# Test

In [ ]:
# 저장된 모델 경로
model_checkpoint = "../models/kobert_finetuning"

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained(model_checkpoint).to('cuda')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [46]:
# 테스트(Test) 데이터셋 평가
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.13740025460720062,
 'eval_model_preparation_time': 0.002,
 'eval_accuracy': 0.9755859375,
 'eval_f1': 0.9856569133677567,
 'eval_precision': 0.983963344788087,
 'eval_recall': 0.9873563218390805,
 'eval_runtime': 10.6301,
 'eval_samples_per_second': 96.33,
 'eval_steps_per_second': 3.01}

In [3]:
# Sliding Window('\t' ver.)
def preprocess(texts):
    text = "[CLS] " + " ".join(str(m) + " [SEP]" for m in texts)
    return text

In [5]:
# 실제 대화 테스트
def predict(chat):
    model.eval()
    tokenized_sent = tokenizer(
        chat,
        add_special_tokens=False,   # 이미 [CLS], [SEP] 추가됨
        max_length=512,             # 문장 최대 길이
        truncation=True,            # 문장이 max_length보다 길면 자름
        padding=True,               # 문장이 max_length보다 짧으면 padding
        return_tensors='pt'         # PyTorch tensor로 반환
    )
    tokenized_sent.to('cuda')
    
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )
        
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)  
    
    if result == 0:
        return '일상 대화 😇'
    elif result == 1:
        return '그루밍 대화 👿'

In [ ]:
test = [
    '오늘 뭐했어?',
    '오늘 그냥 학교 갔다 왔어',
    '오빠는?',
    '나도 오늘 그냥 집에서 쉬었어',
    '근데 너 생각 많이 났어',
    '왜?ㅎㅎㅎ',
    '키스하고 싶어서',
    '아 뭐래~',
    '왜 키스가 어때서',
    '나 아직 미성년자야',
    '아 미안해',
    '그래 자중해~',
    '나 공부하고 올게!!',
    '그래 파이팅!!',
    '오키~',
    '잠만 가기 전에 다리 사진 한번만 보내줘~',
    '아 안돼~',
    '지금 너무 꼴려서..',
    '미안해..',
    '그럼 담에 다리 찍어서 보내주기다~',
    '알았어~'
]


for i in range(len(test)):
    print(f'[Chat]:')
    for j in test[:i+1]:
        print(j)
    print(f'[Pred]: {predict(preprocess(test[:i+1]))}\n\n')

[Chat]:
오늘 뭐했어?
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
키스하고 싶어서
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
키스하고 싶어서
아 뭐래~
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
키스하고 싶어서
아 뭐래~
왜 키스가 어때서
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
키스하고 싶어서
아 뭐래~
왜 키스가 어때서
나 아직 미성년자야
[Pred]: 일상 대화 😇


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠는?
나도 오늘 그냥 집에서 쉬었어
근데 너 생각 많이 났어
왜?ㅎㅎㅎ
키스하고 싶어서
아 뭐래~
왜 키스가 어때서
나 아직 미성년자야
아 미안해
[Pred]: 그루밍 대화 👿


[Chat]:
오늘 뭐했어?
오늘 그냥 학교 갔다 왔어
오빠